In [89]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [90]:
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from scipy import sparse


import pickle5 as pickle
import math
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from collections import defaultdict
from scipy import sparse

%matplotlib inline

In [91]:
# Get dataset from stellargraph
dataset = datasets.Cora()
display(HTML(dataset.description))
# graph, _ = dataset.load(largest_connected_component_only=True, str_node_ids=True)
graph, _ = dataset.load(largest_connected_component_only=True)

In [92]:
nodeFeatures = graph.node_features(nodes=None)
features = sparse.csr_matrix(nodeFeatures)
adj = graph.to_adjacency_matrix(nodes=None)

In [93]:
# make nx graph from scipy matrix
nxGraph = nx.from_scipy_sparse_array(adj, parallel_edges=False, create_using=nx.MultiGraph)

# add features to nx graph
for node_id, node_data in nxGraph.nodes(data=True):
    node_feature = features[node_id].todense()
    node_data["feature"] = np.squeeze(np.asarray(node_feature)) # convert to 1D matrix to array

# make StellarGraph from nx graph
sgGraph = StellarGraph.from_networkx(nxGraph, node_type_default="paper", edge_type_default="cites", node_features="feature")

In [94]:
data_name = 'cora'
hop_count = 0

In [95]:
print("stellargraph original cora graph")
print(graph.info())

print("-----------------------------------")
print("converted cora graph")
print(sgGraph.info())

stellargraph original cora graph
StellarGraph: Undirected multigraph
 Nodes: 2485, Edges: 5209

 Node types:
  paper: [2485]
    Features: float32 vector, length 1433
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5209]
        Weights: all 1 (default)
        Features: none
-----------------------------------
converted cora graph
StellarGraph: Undirected multigraph
 Nodes: 2485, Edges: 5069

 Node types:
  paper: [2485]
    Features: float32 vector, length 1433
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5069]
        Weights: range=[1, 2], mean=1.02762, std=0.163894
        Features: none


In [101]:
# compare node features
f1 = graph.node_features(nodes=None)
f2 = sgGraph.node_features(nodes=None)

print(np.sum(f1))
print(np.sum(f2))

comparison = f1 == f2
equal_arrays = comparison.all()
print(equal_arrays)

45487.0
45487.0
True
